# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer.csv", encoding="UTF-8")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,"大野 あや子,1",女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,"宇多田 貴美子,1",女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,
CS028811000001,"堀井 かおり,1",女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

---
> R-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [8]:
head(df_customer %>% filter(grepl("1$", customer_id)), n=10)

# dplyrでfilter時にlike（部分一致）で絞り込みたいときはgreplを使えばよい
# https://datasciencehenomiti.com/post-613/
# https://qiita.com/naofumi-fujii/items/71f41182a23ded86388b

# ^1$ の $ は正規表現で「直前の文字が行の 末尾 にある場合にマッチ」
# https://murashun.jp/article/programming/regular-expression.html

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS028811000001,"堀井 かおり,1",女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,
CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS028314000011,"小菅 あおい,1",女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5,
CS039212000051,"藤島 恵梨香,1",女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4,
CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
CS004702000041,"野島 洋,0",男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0,
CS041515000001,"栗田 千夏,1",女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F,
CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> R-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [9]:
df_store %>% filter(grepl("横浜市", address))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657
S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044


---
> R-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [10]:
head(df_customer %>% filter(grepl("^[A-F]", status_cd)), n=10)

# 正規表現 [...] 角括弧に含まれるいずれか１文字にマッチ。

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
CS027515000123,田村 咲,1,女性,1965-06-01,53,251-0015,神奈川県藤沢市川名**********,S14027,20150210,E-20100901-E
CS028415000057,山内 瞬,1,女性,1975-08-26,43,246-0035,神奈川県横浜市瀬谷区下瀬谷**********,S14028,20150518,F-20100908-F
CS022414000100,戸田 そら,1,女性,1973-10-19,45,249-0006,神奈川県逗子市逗子**********,S14022,20150926,A-20090814-C
CS004415000388,浅野 ちえみ,1,女性,1973-09-15,45,165-0027,東京都中野区野方**********,S13004,20161008,C-20101008-C
CS019415000117,宮脇 芽以,1,女性,1974-07-10,44,173-0036,東京都板橋区向原**********,S13019,20141114,C-20100720-D
CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
CS024415000108,広末 コウ,1,女性,1969-04-25,49,214-0036,神奈川県川崎市多摩区南生田**********,S14024,20150311,D-20100608-D
CS026515000132,江崎 くるみ,1,女性,1964-10-14,54,251-0045,神奈川県藤沢市辻堂東海岸**********,S14026,20150527,D-20100306-D


---
> R-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [11]:
head(df_customer %>% filter(grepl("[1-9]$", status_cd)), n=10)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
CS006415000279,末永 綾,1,女性,1969-10-03,49,224-0037,神奈川県横浜市都筑区茅ケ崎南**********,S14006,20150713,1-20100519-2
CS001212000098,菊地 りえ,9,不明,1995-01-19,24,210-0842,神奈川県川崎市川崎区渡田東町**********,S13001,20170604,3-20090405-2
CS008615000055,二宮 コウ,1,女性,1953-06-15,65,157-0074,東京都世田谷区大蔵**********,S13008,20150623,5-20100531-8
CS030615000018,島崎 美菜,1,女性,1957-10-16,61,272-0024,千葉県市川市稲荷木**********,S12030,20150925,2-20100705-4


---
> R-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [12]:
head(df_customer %>% filter(grepl("^[A-Z].*[1-9]$", status_cd)), n=10)

# 正規表現 .(ピリオド) 任意の１文字 にマッチ。
# * 直前の文字が ０回以上 繰り返す場合にマッチ。

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
CS002614000180,関口 沙知絵,1,女性,1948-10-19,70,187-0021,東京都小平市上水南町**********,S13002,20170130,C-20100830-7
CS022513000145,河原 美佳,1,女性,1966-02-20,53,249-0005,神奈川県逗子市桜山**********,S14022,20150927,D-20100207-7
CS013615000047,紺野 花,1,女性,1949-03-27,70,275-0021,千葉県習志野市袖ケ浦**********,S12013,20150110,B-20090629-6
CS003415000776,望月 薫,1,女性,1977-01-28,42,201-0002,東京都狛江市東野川**********,S13003,20171020,D-20100903-7
